## Google colab setup

In [ ]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [ ]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  !pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 19.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 7.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

## Imports and setup

In [1]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import time
import spacy
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode

2023-03-24 13:42:41.738072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 13:42:42.250576: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64::/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64:
2023-03-24 13:42:42.250621: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

## Utils

In [3]:
data = pd.read_csv("train_ner.csv")

In [4]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)

    return s

In [5]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [6]:
data["preprocessed"] = data["text"].apply(preprocess)
data.head()

,id,text,label,preprocessed
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...


In [7]:
data["class"] = data["label"].map(lambda x: label_ids[x])
data.head(5)

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...,1


In [8]:
data["label"].value_counts()

OTHER        3649
ABUSE        2768
INSULT       2242
PROFANITY    1294
Name: label, dtype: int64

## Prepare data splits

In [9]:
tok_robert = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")
robert = TFAutoModel.from_pretrained("readerbench/RoBERT-base")

2023-03-24 13:43:05.221248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 13:43:05.221877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 13:43:05.222219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 13:43:05.222502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [10]:
def split_ssl_data(ids_array,mask_array,labels,num_classes,label_percent):
  labeled = None
  unlabeled = None

  for class_idx in range(num_classes):
    class_ids = ids_array[labels==class_idx]
    class_mask = mask_array[labels==class_idx]
    sz = int(label_percent * class_ids.shape[0])

    labels_reduced = labels[labels==class_idx][:sz]
    labeled_ids, unlabeled_ids = class_ids[:sz], class_ids[sz:]
    labeled_mask, unlabeled_mask = class_mask[:sz], class_mask[sz:]

    if not labeled:
      labeled = (labeled_ids, labeled_mask, labels_reduced)
      unlabeled = (unlabeled_ids, unlabeled_mask)
    else:
      labeled = (
          np.concatenate([labeled[0],labeled_ids]),
          np.concatenate([labeled[1],labeled_mask]),
          np.concatenate([labeled[2],labels_reduced])
      )
      unlabeled = (
          np.concatenate([unlabeled[0],unlabeled_ids]),
          np.concatenate([unlabeled[1],unlabeled_mask]),
      )

  return labeled, unlabeled


In [11]:
def preprocess_robert(x):
  t = tok_robert(x,padding="max_length",max_length=96,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

def map_func2(input_ids, masks):
  return {'input_ids': input_ids, 'attention_mask': masks}

def prepare_ds(filename,batch_size=64,label_percent=0.8):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  return res_ds

In [12]:
def prepare_train_ds(filename,batch_size=16,label_percent=0.05):
  df = pd.read_csv(filename)
  df = df.sample(frac=1)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  labeled, unlabeled = split_ssl_data(ids_array,mask_array,labels,len(label_ids),label_percent)
  lres = None
  ures = None
  ids,mask,labels = labeled
  for i in range(0,len(ids),200):
    end = min(i+200,len(ids))
    embeds = robert(input_ids=ids[i:end],attention_mask=mask[i:end],training=False).last_hidden_state.numpy()
    if i==0:
      lres = embeds
    else:
      lres = np.concatenate([lres,embeds])
  ids,mask = unlabeled
  for i in range(0,len(ids),200):
      end = min(i+200,len(ids))
      embeds = robert(input_ids=ids[i:end],attention_mask=mask[i:end],training=False).last_hidden_state.numpy()
      if i==0:
        ures = embeds
      else:
        ures = np.concatenate([ures,embeds])

  labeled_ds = tf.data.Dataset.from_tensor_slices((lres,labels))
  labeled_ds = labeled_ds.shuffle(len(labeled_ds)).batch(batch_size).repeat()
  unlabeled_ds = tf.data.Dataset.from_tensor_slices(ures)
  unlabeled_ds = unlabeled_ds.shuffle(len(unlabeled_ds)).batch(4*batch_size).repeat()

  return labeled_ds, unlabeled_ds

In [13]:
labeled_ds, unlabeled_ds= prepare_train_ds("train_ner.csv")
test_ds = prepare_ds("test_ner.csv")
val_ds = prepare_ds("validation_internal_ner.csv")

## Model definition and declaration

In [14]:
class MixMatch(tf.keras.Model):
  def __init__(self,num_classes=4,**kwargs):
    super(MixMatch, self).__init__(**kwargs)
    self.num_classes = num_classes

    self.cls_head = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(10,dropout=0.2),
            merge_mode="ave"
        ),
        tf.keras.layers.Dense(units=self.num_classes,activation="softmax")
    ])


  def call(self, inputs, training):
    
    out = self.cls_head(inputs,training=training)
    return out
  
class Augment(tf.keras.Model):
  def __init__(self,**kwargs):
    super(Augment,self).__init__(**kwargs)
    self.augment = tf.keras.layers.GaussianNoise(stddev=1)

  def call(self,inputs,training):
    return self.augment(inputs,training=training)

class BERTEncoder(tf.keras.Model):
  def __init__(self,bert_model,**kwargs):
    super(BERTEncoder, self).__init__(**kwargs)
    self.bert = bert_model
    self.bert.trainable = False
    

  def call(self, inputs):
    ids, mask = inputs
    embeds = self.bert(input_ids=ids, attention_mask=mask).last_hidden_state
    return embeds


In [22]:
cls_head = MixMatch()
encoder = BERTEncoder(bert_model=robert)
aug = Augment()
ids_input = tf.keras.Input(shape=(96,),dtype=tf.int32)
mask_input = tf.keras.Input(shape=(96,),dtype=tf.int32)
embeds = encoder([ids_input,mask_input])
model_output = cls_head(aug(embeds))
model = tf.keras.Model(inputs=[ids_input,mask_input],outputs=model_output)


## Train and evaluate

In [23]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.001)
entropy = tf.keras.losses.CategoricalCrossentropy()
mse = tf.keras.losses.MeanSquaredError()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [24]:
def weight_schedule(t, init_period, final_period, max_period):

  if t < init_period:
    x = t / init_period
    return 2.71 ** (-5*(x-1)**2)
  elif t < max_period - final_period:
    return 1.0
  else:
    x = (t - max_period + final_period) / final_period
    return 1 - 2.71 ** (-5*(x-1)**2)

In [25]:
epochs = 30
K = 2
steps_per_epoch = 151
max_period = epochs * steps_per_epoch
alpha = 10
unsup_weight = tf.Variable(1.0)
init_period = 700
final_period = 700
blend = tf.constant(1.0)

In [26]:
@tf.function
def sharpen(x,T=0.5):
  power = 1/T
  x = x**power
  s = tf.expand_dims(tf.reduce_sum(x,axis=1),axis=-1)
  return x/s

@tf.function
def train_step(x,y,xu,unsup_weight,blend):

  x_aug = aug(x,training=True)

  r_xu = tf.repeat(xu,K,axis=0)
  cnt = tf.shape(x_aug)[0]

  xu_aug = aug(r_xu, training=True)

  pred_unsup = cls_head(xu_aug,training=False)

  pred_unsup = tf.reshape(pred_unsup,(-1, K, 4))

  pred_unsup = tf.repeat(sharpen(tf.reduce_mean(pred_unsup,axis=1)),K,axis=0)

  y_ext = tf.one_hot(y,4)

# (x_aug, y_ext) , (xu_aug, pred_unsup)

  x_total = tf.random.shuffle(tf.concat([x_aug,xu_aug],axis=0),seed=1)
  total = tf.shape(x_total)[0]
  y_total = tf.random.shuffle(tf.concat([y_ext, pred_unsup],axis=0),seed=1)

  first = tf.gather(x_total,range(cnt),axis=0)
  first_labels = tf.gather(y_total,range(cnt),axis=0)


  second = tf.gather(x_total,range(cnt,total),axis=0)
  second_labels = tf.gather(y_total,range(cnt,total),axis=0)

  a = blend * x_aug + (1-blend) * first
  a_target = blend * y_ext + (1-blend) * first_labels

  b = blend * xu_aug + (1-blend) * second
  b_target = blend * pred_unsup + (1-blend) * second_labels

  with tf.GradientTape() as tape:
      
      a_pred = cls_head(a,training=True)
      b_pred = cls_head(b,training=True)

      loss = entropy(a_target, a_pred) + unsup_weight * mse(b_target, b_pred) / 4
      
  grads = tape.gradient(loss, cls_head.trainable_weights)
  optim.apply_gradients(zip(grads, cls_head.trainable_weights))

  return loss


@tf.function
def test_step(x, y):
    wpred = model([x["input_ids"],x["attention_mask"]], training=False)
    val_acc_metric.update_state(y, wpred)
    true_hot = tf.one_hot(y, 4)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)

In [27]:
max_val_acc = 0.0
best_weights = None
l_iter = iter(labeled_ds)
u_iter = iter(unlabeled_ds)
t_step = 0

for epoch in range(epochs):
  print(f"Epoch {epoch}")
  start_time = time.time()

  for step in range(steps_per_epoch):
    x, y = next(l_iter)
    xu = next(u_iter)
    lam = np.random.beta(alpha, alpha)
    lam = max([lam,1-lam])
    blend = tf.constant(lam)
    unsup_val = weight_schedule(t_step, init_period, final_period, max_period)
    unsup_weight.assign(4.0 * unsup_val)
    
    loss = train_step(x,y,xu,unsup_weight,blend)

    if step % 50 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )
    t_step+=1
  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = model.get_weights()

  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))
model.set_weights(best_weights)


Epoch 0
Training loss (for one batch) at step 0: 1.3761
Training loss (for one batch) at step 50: 1.1941
Training loss (for one batch) at step 100: 1.1769
Training loss (for one batch) at step 150: 1.1775
Validation acc: 0.492 precision: 0.629 recall: 0.260 f1_micro: 0.368 f1_macro: 0.172
Time taken: 23.72s
Epoch 1
Training loss (for one batch) at step 0: 1.1100
Training loss (for one batch) at step 50: 1.2021
Training loss (for one batch) at step 100: 1.1366
Training loss (for one batch) at step 150: 1.1749
Validation acc: 0.528 precision: 0.623 recall: 0.398 f1_micro: 0.486 f1_macro: 0.303
Time taken: 16.04s
Epoch 2
Training loss (for one batch) at step 0: 1.1373
Training loss (for one batch) at step 50: 1.0992
Training loss (for one batch) at step 100: 0.9356
Training loss (for one batch) at step 150: 1.2303
Validation acc: 0.529 precision: 0.628 recall: 0.414 f1_micro: 0.499 f1_macro: 0.318
Time taken: 16.69s
Epoch 3
Training loss (for one batch) at step 0: 1.1138
Training loss (fo

In [28]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.556 precision: 0.623 recall: 0.439 f1_micro: 0.515 f1_macro: 0.399
